In [1]:
!pip install gurobipy  # install gurobipy, if not already installed

     |████████████████████████████████| 11.5 MB 5.2 MB/s 


In [2]:
from ast import Pass
from re import M
from turtle import shape
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [3]:
def complete_graph(graph):
        graph_original = graph
        for e in graph_original.edges():
            graph_original[e[0]][e[1]]['length'] = int(graph_original[e[0]][e[1]]['length'])
        graph_complete = graph_original.copy()
        nodes = list(graph_original.nodes())
        paths = {}
        paths = dict(nx.all_pairs_dijkstra_path(graph_original, weight = 'length'))
        for i in nodes:
            for j in nodes:
                if i != j and not (i, j) in graph_complete.edges():
                    graph_complete.add_edge(i, j)
                    graph_complete[i][j]['name'] = '{}to{}'.format(i, j)
                    graph_complete[i][j]['length'] = 0
                    paths[(i, j)] = paths[i][j]
                    for k in range(len(paths[i, j]) - 1):
                        graph_complete[i][j]['length'] += graph_original[paths[i][j][k]][paths[i][j][k + 1]]['length']
                elif i != j:
                    paths[(i, j)] = [i, j]

        return graph_complete

In [4]:
temp = np.zeros(shape=(5))
temp[0]

0.0

In [5]:
class mid_mile:

    def __init__(self, K, L, M, R):

        #self.graph = graph
        self.K = K
        self.L = L                                       # no of legs
        self.M = M
        self.R = R                                       # let the routes be defined by their numbers - i.e. R = 5 means there are 5 routes named: r0, r1, ...
        self.V = np.zeros(shape=(self.K))           # the demand of each commodity K in route R
        self.C = None                                         
        self.F = np.zeros(shape=(self.L, self.M))
        self.A = np.zeros(shape=(self.L, self.M))
        self.B = np.zeros(shape=(self.L, self.M))
        self.Qmax = np.zeros(shape=(self.L, self.M))
        self.Qmin = np.zeros(shape=(self.L, self.M))

        self.model = gp.Model()
        
    def add_variables(self):
        lm = []

        # decision variables: xr, ylm, vlm, flm
        for a in range(self.L):
            for b in range(self.M):
                lm.append((a,b))
        
        kr = []
        for r in range(self.R):
            for k in range(self.K):
                kr.append((k,r))

        lm = gp.tuplelist(lm)
        kr = gp.tuplelist(kr)

        self.x = self.model.addVars(kr, vtype=GRB.BINARY, name="x")
        self.y = self.model.addVars(lm, vtype=GRB.BINARY, name="y")
        self.v = self.model.addVars(lm, vtype=GRB.INTEGER, name="v")
        self.f = self.model.addVars(lm, vtype=GRB.INTEGER, name="f")

        self.model.update()

    def add_init_constraints(self):
        self.add_constraints1()
        self.add_constraints2()
        self.add_constraints3()
        self.add_constraints4()
        self.add_constraints5()
        #self.obj_constraint()

    def add_constraints1(self):
        # sigma xr = 1
        for k in range(self.K):
            lhs = 0
            for i in range(self.R):
                lhs += self.x[k,i]
            self.c1 = self.model.addConstr(lhs == 1)
        
        self.model.update()

    def add_constraints2(self):
        # summation of shipments should tally
        
        for i in range(self.L):
            lhs = 0
            for j in range(self.M):
                lhs += self.v[i, j]
            
            rhs = 0
            for k in range(self.K):
                for r in range(self.R):
                    rhs += self.V[k]*self.x[k,r]
            
            c = self.model.addConstr(lhs == rhs)
        
        self.model.update()

    def add_constraints3(self):
        # upper limit
        for l in range(self.L):
            for m in range(self.M):
                lhs = self.v[l, m]
                rhs = self.Qmax[l][m]
                c = self.model.addConstr(lhs <= rhs)

        # lower limit
        for i in range(self.L):
            for j in range(self.M):
                lhs = self.v[l, m]
                rhs = self.Qmin[l][m]
                c = self.model.addConstr(lhs >= rhs)
        
        self.model.update()

    def add_constraints4(self):
        for l in range(self.L):
            for m in range(self.M):
                lhs = self.y[l, m]
                c = self.model.addConstr(lhs <= 1)
        
        self.model.update()
    
    def add_constraints5(self):
        for l in range(self.L):
            for m in range(self.M):
                lhs = self.f[l, m]
                rhs = self.F[l][m]
                c = self.model.addConstr(lhs <= rhs)
        
        self.model.update()


    # def obj_constraint(self):
    #     lhs = 0
    #     for e in self.edges:
    #         lhs += self.graph[e[0]][e[1]]['length'] * self.x[a, e[0], e[1]]
    #     #this constraint ensures that c is atleast equal to the max of the walks of the agents
    #     print(self.c)
    #     self.model.addConstr(lhs <= self.c[0], name="obj_agent{}".format(a))
        
    #     self.model.update()


    def objective(self):
        term1 = 0
        for k in range(self.K):
            for i in range(self.R):
                term1 += self.C[i] * self.x[k,i]
        
        term2 = 0
        for l in range(self.L):
            for m in range(self.M):
                term2 += self.A[l][m]*self.f[l,m] + self.B[l][m]*self.v[l,m]
        
        obj = term1 + term2

        self.model.setObjective(obj, GRB.MINIMIZE)

    def process(self):
        self.status = self.model.optimize()

    def get_solution(self):
        Pass

In [6]:
# Input model parameter values

K = 1
R = 4
V = np.zeros(shape=(K))
V[0] = 10
C = [5, 10, 15, 25]
L = 6
M = 1                       # only one mode of transport

F = np.zeros(shape=(L, M))
for i in range(L):
    for j in range(M):
        F[i][j] = 5*i + j

A = np.zeros(shape=(L, M))
for i in range(L):
    for j in range(M):
        A[i][j] = 2*i + 3*j

B = np.zeros(shape=(L, M))
for i in range(L):
    for j in range(M):
        B[i][j] = 2*i + 3*j

Qmax = np.zeros(shape=(L, M))
for i in range(L):
    for j in range(M):
        Qmax[i][j] = 100*i + 3*j

Qmin = np.zeros(shape=(L, M))
for i in range(L):
    for j in range(M):
        Qmin[i][j] = 1*i + 3*j

In [7]:
test = mid_mile(K, L, M, R)

test.V = V
test.C = C

test.F = F
test.A = A
test.B = B
test.Qmax = Qmax
test.Qmin = Qmin

Restricted license - for non-production use only - expires 2023-10-25


In [8]:
test.add_variables()
test.add_init_constraints()
test.objective()
test.process()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 31 rows, 22 columns and 58 nonzeros
Model fingerprint: 0xaf7ed668
Variable types: 0 continuous, 22 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Presolve removed 28 rows and 18 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
